In [39]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? u
Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Train

## SetUp directories

In [12]:
import os
# Location of models
model_directory = '../model'
if(not os.path.exists(model_directory)):
    os.makedirs(model_directory)

NameError: name 'model_directory' is not defined

## Parameters to set: very important!

In [1]:

train_path = '../data/trainFile.txt'

# Dimension of vectors
dim_vec = 300

# Number of threads used during training, should be equal to number of cores if one wants to minimize training time 
threads = 4

In [2]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y")
global_path = "../models/"
bin_path = ".bin"
model_path = ".model"
txt_path = ".txt"
path_ft_bin =  global_path + "ft_"+ dt_string + bin_path
path_ft_txt =  global_path + "ft_"+ dt_string + txt_path
path_w2v =  global_path + "w2v_"+ dt_string + model_path
model_path_glove = global_path + "gloVe_"+ dt_string + txt_path

## Fasttext

In [43]:
import fasttext
model = fasttext.train_unsupervised(train_path,thread=threads,epoch=9,dim=dim_vector)

In [44]:
model.save_model(path_ft)

We only want to save the vectors at it will cost less storage space

In [47]:
from fasttext import load_model

# original BIN model loading
f = load_model(path_ft_bin)
lines=[]

# get all words from model
words = f.get_words()

with open(path_ft_txt,'w') as file_out:
    # the first line must contain number of total words and vector dimension
    file_out.write(str(len(words)) + " " + str(f.get_dimension()) + "\n")

    # line by line, you append vectors to VEC file
    for w in words:
        v = f.get_word_vector(w)
        vstr = ""
        for vi in v:
            vstr += " " + str(vi)
        try:
            file_out.write(w + vstr+'\n')
        except:
            pass

In [49]:
import subprocess
subprocess.run(["rm", path_ft_bin])

CompletedProcess(args=['rm', '../models/ft_14-05-2020.bin'], returncode=0)

## Word2Vec

In [6]:
with open(train_path) as f:
    corpus = f.readlines()
res = []
for sent in corpus: 
    sent = sent[0:len(sent)-1]
    res.append(sent.split(" "))

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(res, size=300,window=5,negative=10, alpha=0.01,iter=9,
                 min_count=5, workers=4,sg=1,compute_loss=True)


unable to import 'smart_open.gcs', disabling that module


In [8]:
from gensim.test.utils import common_texts, get_tmpfile
path = get_tmpfile(path_w2v)
model.save(path_w2v)

## Glove 
Adjust Path to your local GloVe Repo
https://github.com/stanfordnlp/GloVe

In [4]:
import subprocess
glove_path = "/home/c3dric/Programming/GloVe/"
glove_corpus_path = glove_path + "press_all_glove.txt"
glove_script = glove_path + "demo.sh"
glove_vectors = glove_path + "vectors.txt"

In [5]:
# Copy file into glove directory
subprocess.run(["cp", train_path, glove_corpus_path])
# Start Training
subprocess.run(["sh", glove_script],cwd=glove_path)
# Transform File into w2v format (simplier to use during evlauation)
number_of_lines = subprocess.check_output("wc -l " + glove_vectors , shell=True).split()[0].decode('utf-8')
subprocess.check_output("sed -i '1i " + number_of_lines + " " + str(dim_vec) + "' " + glove_vectors,shell=True)
# Copy file into local models
subprocess.run(["cp", glove_vectors, model_path_glove])

CompletedProcess(args=['cp', '/home/c3dric/Programming/GloVe/vectors.txt', '../models/gloVe_14-05-2020.txt'], returncode=0)